In [ ]:
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Harvard Med School Question ##

In [ ]:
['Disease'] * 10

In [ ]:
def create_branch(a, b, n):
    t = Table().with_column('Status', [a] * n)
    t.append_column('Test Result', b)
    return t

In [ ]:
def create_population(prior_prob_of_disease, n):
    t = create_branch('Disease', '+', round(prior_prob_of_disease * n))
    t.append(create_branch('No disease', '+', round((1-prior_prob_of_disease) * 0.05 * n)))
    t.append(create_branch('No disease', '-', round((1-prior_prob_of_disease) * 0.95 * n)))
    return t

In [ ]:
create_population(0.001, 100000).pivot('Test Result', 'Status')

## Broste Thesis Data ##

In [ ]:
summary = Table(['Age', 'Condition', 'Total', 'Deaths', 'CHD Deaths']).with_rows([
    ['0-34',  'Diet',    1367, 3, 0],
    ['35-44', 'Diet',    728, 3, 0],
    ['45-54', 'Diet',    767, 14, 4],
    ['55-64', 'Diet',    870, 35, 7],
    ['65+',   'Diet',    953, 190, 42],
    ['0-34',  'Control', 1337, 7, 1],
    ['35-44', 'Control', 731, 4, 1],
    ['45-54', 'Control', 816, 16, 4],
    ['55-64', 'Control', 896, 33, 12],
    ['65+',   'Control', 958, 162, 34],   
])
summary

In [ ]:
np.arange(12) < 3

In [ ]:
subjects = Table(['Age', 'Condition', 'Participated', 'Died'])
for row in summary.rows:
    i = np.arange(0, row.item('Total'))
    t = Table().with_columns('Died', i < row.item('Deaths'))
    t.append_column('Age', row.item('Age'))
    t.append_column('Condition', row.item('Condition'))
    t.append_column('Participated', True)
    subjects.append(t)
subjects

In [ ]:
subjects.group(['Age', 'Condition'], sum)

In [ ]:
subjects.group('Condition', sum)

In [ ]:
subjects.drop('Age').group('Condition', sum)

In [ ]:
def hazard_rate(counts):
    return counts.item('Died sum') / counts.item('Participated sum')

def rate_difference(t):
    counts = t.drop('Age').group('Condition', sum)
    return abs(hazard_rate(counts.row(1)) - hazard_rate(counts.row(0)))

rate_difference(subjects)

In [ ]:
rate_difference(subjects.where('Age', '0-34'))

In [ ]:
rate_difference(subjects.where('Age', '65+'))

In [ ]:
def test(t):
    observed = rate_difference(t)
    repetitions = 200

    stats = make_array()
    for i in np.arange(repetitions):
        simulated_results = t.select('Died').sample(with_replacement=False).column('Died')
        simulated_outcomes = t.with_column('Died', simulated_results)
        simulated_stat = rate_difference(simulated_outcomes)
        stats = np.append(stats, simulated_stat)

    # Find the empirical P-value:
    p = np.count_nonzero(stats >= observed) / repetitions
    
    print('Observed absolute difference in hazard rates:', observed)
    print('P-value:', p)

test(subjects)

In [ ]:
subjects.group('Age').column('Age')

In [ ]:
for age in subjects.group('Age').column('Age'):
    print('Ages', age)
    test(subjects.where('Age', age))